In [1]:
quan_file = "/Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/quantification/bedtools_quantification.matrix"
df = pd.read_table(quan_file)
df.head()

,brainadult1,brainadult2,brainadult3,brainadult4,brainadult5,brainadult6,brainfetus1,brainfetus2,brainfetus3,heartadult1,heartadult2,heartfetus1,heartfetus2,heartfetus3,liveradult1,liverfetus1,liverfetus2,liverfetus3,lungadult1,lungadult2,lungadult3,lungadult4,lungfetus1,lungfetus2,stomachadult1,stomachadult2,stomachfetus1,stomachfetus2
chr1:14421-14890,2.165741,4.142653,3.038712,2.454334,2.644958,1.019826,0.417069,0.353146,0.318406,3.452908,1.790184,0.473559,0.310598,0.289530,4.240557,0.331896,0.307644,0.282735,4.207665,2.046219,1.558267,1.853059,0.369157,0.283512,2.287013,2.316979,0.372790,0.360427
chr1:17663-17863,1.031296,1.056552,1.066083,1.304711,1.028059,1.043702,0.437480,0.477354,0.493127,1.058136,1.148424,0.657301,0.607320,0.616489,1.136681,0.590592,0.453253,0.495092,1.255688,1.000000,0.925523,1.208643,0.436171,0.609198,0.919379,1.000000,0.455636,0.558357
chr1:90808-91060,1.273221,1.894050,1.183564,1.053741,1.066480,1.136372,1.020010,1.000000,1.000000,1.138419,1.029676,0.973155,1.000000,1.000000,1.000000,1.000000,1.000000,0.948803,1.121756,1.000000,1.036662,1.000000,1.000000,1.000000,1.243586,1.583282,0.957760,1.000000
chr1:135063-135436,1.264061,2.370550,1.432386,1.780612,2.100649,0.988808,1.037050,0.958002,1.019612,2.589775,1.112186,1.035935,1.049433,0.720308,0.979255,0.870499,1.108570,1.083493,3.199150,1.622437,1.513538,1.447493,1.262229,0.866134,1.423173,0.748138,0.910111,0.984670
chr1:137651-137861,1.112516,2.984409,1.024523,1.057122,1.152782,0.877306,0.978093,0.975248,0.905007,1.941248,0.770685,0.939775,0.870058,0.576519,0.723627,0.716209,0.777056,0.711314,1.438323,0.935368,1.456689,1.000000,0.837374,0.576357,1.000000,0.590925,0.625697,0.840137


In [12]:
work_dir = "/Data_Resource/m6A-seq/com_fetus_vs_adult/diff_methylation/overlap_exomePeak-MeTDiff/mainCaller_MeTDiff"
metdiff_list = glob.glob("%s/*.bed6" % work_dir)
for bed6 in metdiff_list:
    #
    tissue = os.path.basename(bed6).split("_")[0]
    selected_col = [x for x in df.columns if tissue in x]
    df_sub = df[selected_col]
    #
    lines = []
    names = ",".join(["#chr,start,end"] + df_sub.columns.tolist())
#     for m6a, values in df_sub.iterrows():
#         chr_name = m6a.split(":")[0]
#         start = m6a.split(":")[1].split("-")[0]
#         end = m6a.split(":")[1].split("-")[1]
#         lines.append("%s\t%s\t%s\t%s\n" % (chr_name, start, end, ",".join([str(x) for x in values])))
#     # 
    result_dir = "/Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/quantification/by_tissue"
#     with open("%s/%s.bed" % (result_dir, tissue), 'w') as fw:
#         fw.write(names + "\n")
#         fw.writelines(lines)
    intersect_each(bed6, result_dir, work_dir, names)

In [8]:
def intersect_each(bed6, result_dir, work_dir, names):
    tissue = os.path.basename(bed6).split("_")[0]
    quan_bed = "%s/%s.bed" % (result_dir, tissue)
    # maincaller quantification matrix
    outbed = os.path.join(work_dir, "%s_quan.bed" % (tissue))
    with open(outbed, 'w') as fw:
        fw.write("#" + names + "\n")
    os.system("bedtools intersect -a %s -b %s -wa | sort -k1,1 -k2,2n | uniq >> %s" % (quan_bed, bed6, outbed))

           lungadult1          lungadult2         ...                  lungfetus1        lungfetus2\n
0  3.2390371654136096    1.54827435735144         ...          1.0735286028174398   0.707059599409348
1   5.696952390736611  2.2883575645507204         ...            1.44700566341156  1.0312842040435402
2  2.7399632473754503  1.6424878049090201         ...            1.42318348242003  1.0343785360562199
3    4.03890810499121  2.2606242748145897         ...           0.923454715014819   0.682658907234561
4    4.14220773538656  1.9222407516451299         ...            1.08969618368537  0.8639951397869681

[5 rows x 6 columns]


In [ ]:
for tissue in ["brain", "heart", "liver", "lung", "stomach"]:
    ...:     m6a_list=[]
    ...:     cols = ["m6a"] + [x for x in df.columns if tissue in x]
    ...:     df_sub = df[cols]
    ...:     adult_cols = [x for x in cols if "adult" in x]
    ...:     fetal_cols = [x for x in cols if "fetus" in x]
    ...:     for i, values in df_sub.iterrows():
    ...:         adult_values = [values[x] for x in adult_cols]
    ...:         fetal_values = [values[x] for x in fetal_cols]
    ...:         adult_mt = [x for x in adult_values if x >= 5]
    ...:         fetal_mt = [x for x in fetal_values if x >= 5]
    ...:         if len(adult_mt) >= math.ceil(len(adult_values)/2):
    ...:             if len(fetal_mt) >= math.ceil(len(fetal_values)/2):
    ...:                 m6a_list.append(values["m6a"])
    ...:     result_file = "../by_tissue/%s.matrix" % tissue
    ...:     with open(result_file, 'w') as fw:
    ...:         fw.writelines(["%s\n" % x for x in m6a_list])
